In [2]:
import os
import sys
import requests
from config import API_KEY
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO

'''
- https://developer.tomtom.com/search-api/documentation/search-service/points-of-interest-search

'''

'\n- https://developer.tomtom.com/search-api/documentation/search-service/points-of-interest-search\n\n'

In [3]:
stream = os.popen('curl ipinfo.io/loc')
lat, lon = stream.read()[:-len("/n")].split(",") #Incorrect!

lat, lon = 49.266174286260664, -123.24245297485056 #Hardcoded Home
# lat, lon = 49.28306128007426, -123.11922094511674

print(f"Current location: {lat} {lon}") 

def summarize_poi_search(res):
    query_res = res.json()['results']
    if len(query_res) > 0:
        for qres in query_res:
            name = qres['poi']['name']
            address = qres['address']['freeformAddress']
            dist = round(qres['dist']/1e3, 2) #in KM
            print(f"Name: {name} \t {address} \t Distance: {dist} km")
    

Current location: 49.266174286260664 -123.24245297485056


In [5]:
baseURL = "api.tomtom.com"
versionNumber = str(2)

query = "Tim H"
poi_category = "food"
ext = "json"
radius = 3000
lim= 50
typeahead=True

def poi_search(query):
    res = requests.get(f"https://{baseURL}/search/{versionNumber}/poiSearch/{query}.{ext}?key={API_KEY}&lat={lat}&lon={lon}&radius={radius}&countrySet=CAN&typeahead={typeahead}&limit={lim}")
    summarize_poi_search(res)
    return res.json()

'''
Returns all points of interest within a specified radius of a given lat-lon
'''
def nearby_search():
    res = requests.get(f"https://{baseURL}/search/{versionNumber}/nearbySearch/.{ext}?key={API_KEY}&limit={lim}&lat={lat}&lon={lon}&radius={radius}&countrySet=CAN")
    summarize_poi_search(res)
    return res.json()

# nearby_search()
res = poi_search(query)

Name: Tim Hortons 	 2424 Main Mall, Vancouver BC V6T 1Z4 	 Distance: 0.81 km
Name: Tim Hortons 	 2033 Agricultural Road, Vancouver BC V6T 	 Distance: 0.86 km
Name: Tim Hortons 	 4401 West 10th Avenue, Vancouver BC V6R 2H8 	 Distance: 2.63 km


# Routes Endpoint

journey = f"{lat},{lon}:{result['position']['lat']},{result['position']['lon']}"
# travel_mode = "car"
travel_mode = "pedestrian"
res = requests.get(f"https://{baseURL}/routing/1/calculateRoute/{journey}/json?key={API_KEY}&travelMode={travel_mode}")
# res.json()['routes']

# Interactive Map

In [6]:
def get_icon(category):
    if category == 'food':
        return folium.Icon(color="green", icon="cutlery", prefix='fa')
    else:
        raise ValueError(f'Invalid category: {category}')
        
'''
https://towardsdatascience.com/how-to-make-stunning-interactive-maps-with-python-and-folium-in-minutes-e3aff3b0ed43
'''
def generate_popup(r_id, result_item):
    lat, lon = result_item['position']['lat'], result_item['position']['lon']
    name = result_item['poi']['name']
    return f'''<strong> {r_id} {name} </strong> <br> {lat},{lon}'''

def get_travel_time(res):
    print(f"Travel time(mins): {int(res.json()['routes'][0]['summary']['travelTimeInSeconds'])//60}")

In [9]:
import folium
fmap = folium.Map(location=[lat, lon], zoom_start=16, width="100%", height="50%")

folium.Marker(
    location=[lat, lon], icon=folium.Icon(color="red",icon="home", prefix='fa')
).add_to(fmap)

#Add a radius
folium.Circle(location=[lat, lon],
    color='red',
    radius=radius
).add_to(fmap)

res = poi_search(query)
for r_id, result in enumerate(res['results']):
    folium.Marker(
        location = [result['position']['lat'], result['position']['lon']],
#         popup = result['poi']['name'],
        popup = generate_popup(r_id, result),
        icon = get_icon(poi_category)
    ).add_to(fmap)
    
    
#Draw a line to one of the locations
target_pos = res['results'][0]['position']
lat_2, lon_2 = target_pos['lat'], target_pos['lon']
# lat_2, lon_2 = 49.266174286260664, -123.24245297485056
journey = f"{lat},{lon}:{lat_2},{lon_2}"

# travel_mode = "car"
# res = requests.get(f"https://{baseURL}/routing/1/calculateRoute/{journey}/json?key={API_KEY}&travelMode={travel_mode}")
# get_travel_time(res)
# path_points = [(loc['latitude'], loc['longitude']) for loc in res.json()['routes'][0]['legs'][0]['points']] #Get first available route
# folium.PolyLine(path_points,color="blue", weight=5.5, opacity=0.5).add_to(fmap)

travel_mode = "pedestrian"
#https://developer.tomtom.com/routing-api/documentation/routing/common-routing-parameters
travel_mode = "bus"
res = requests.get(f"https://{baseURL}/routing/1/calculateRoute/{journey}/json?key={API_KEY}&travelMode={travel_mode}")
get_travel_time(res)
path_points = [(loc['latitude'], loc['longitude']) for loc in res.json()['routes'][0]['legs'][0]['points']] #Get first available route
folium.PolyLine(path_points,color="green", weight=5.5, opacity=0.5).add_to(fmap)

fmap

Name: Tim Hortons 	 2424 Main Mall, Vancouver BC V6T 1Z4 	 Distance: 0.81 km
Name: Tim Hortons 	 2033 Agricultural Road, Vancouver BC V6T 	 Distance: 0.86 km
Name: Tim Hortons 	 4401 West 10th Avenue, Vancouver BC V6R 2H8 	 Distance: 2.63 km
Travel time(mins): 6


# Extended Search API

In [248]:
baseURL = "api.tomtom.com"
versionNumber = str(2)
query = "pizza"
ext = "json"
radius = 4000
lim= 100

def plot_image(img_res):
    img = Image.open(BytesIO(img_res.content))
    plt.figure()
    plt.imshow(img)
    plt.axis('off')
    plt.show()
    plt.close()
    
def get_photo_res(pid):
    photo_res = requests.get(f"https://{baseURL}/search/{versionNumber}/poiPhoto?key={API_KEY}&id={pid}")
    return photo_res

def visualize_popular_hours(activity_series):
    for day in activity_series:
        print(day)
#         print(day['dayOfWeek'])
    

def extended_poi_search(poi_name, poi_id, plot_ims=False):
    print(f"******** Searching for [{poi_name}] ********")
    res = requests.get(f"https://{baseURL}/search/{versionNumber}/poiDetails.{ext}?key={API_KEY}&id={poi_id}").json()
    print(res['result'].keys())
    if plot_ims:
        photo_res_obs = [get_photo_res(photo['id']) for photo in res['result']['photos']]
        for pres in photo_res_obs:
            plot_image(pres)
    
    #Print reviews
    if 'reviews' in res['result']:
        print(f"*** REVIEWS *** \n")
        for rev in res['result']['reviews']:
            date, rev_text = rev['date'], rev['text']
            print(f"{date} - {rev_text}")
    if 'priceRange' in res['result']:
        print(f"*** PRICE RANGE *** \n")
        print(res['result']['priceRange']['label'])
    if 'popularHours' in res['result']:
        print(f"*** Popular Hours *** \n")
        visualize_popular_hours(res['result']['popularHours'])

        
'''
USAGE
'''
poi_res = poi_search(query)
poi = poi_res['results'][5]
poi_id, poi_name = poi['id'], poi['poi']['name']
extended_poi_search(poi_name, poi_id, plot_ims=False)

Name: Freshslice Pizza 	 2166 Western Parkway, Vancouver BC V6T 1V6 	 Distance: 0.04 km
Name: University Plaza Pizza 	 5728 University Boulevard, Vancouver BC V6T 1K6 	 Distance: 0.05 km
Name: Pizza Garden 	 5780 University Boulevard, Vancouver BC V6T 1K6 	 Distance: 0.07 km
Name: Pie R Squared 	 6138 Toronto Road, Vancouver BC V6T 	 Distance: 0.24 km
Name: Uncle Fatih's Pizza-UBC 	 University Boulevard, Vancouver BC V6T 0C4 	 Distance: 0.33 km
Name: Pie R Squared 	 6138 Student Union Boulevard, Vancouver BC V6T 1Z1 	 Distance: 0.57 km
Name: Mercante 	 6488 University Boulevard, Vancouver BC V6T 	 Distance: 0.96 km
Name: Pizza 222 Express 	 University Boulevard, Vancouver BC V6T 	 Distance: 1.14 km
Name: Virtuous Pie 	 3339 Shrum Lane, Vancouver BC V6T 1W5 	 Distance: 1.45 km
Name: Pizza Hut 	 4540 West 10th Avenue, Vancouver BC V6R 2J1 	 Distance: 2.31 km
******** Searching for [Pie R Squared] ********
dict_keys(['description', 'rating', 'priceRange', 'photos', 'reviews', 'popularHour